- trf
    - ca37: distance=passthrough
    - fa15: distance=pwr
- imp
    - 2474: tune=False
    - 20cf: tune=True
- clf
    - 2c44: ensemble=True, time=20
    - 25f2: ensemble=False, time=20

In [1]:
from LiveAMP import *
# run_amp(137)
self = get_stack(137, '_2024_05_07')

creating stack.pkl: Elapsed time: 43.4536 seconds
